## Set up Bigquery Database

In [13]:
import time
from datetime import datetime
import pytz
from airflow.decorators import dag, task
from airflow.utils.task_group import TaskGroup
from airflow.models import DAG
import pandas as pd
import pandas_gbq
from bs4 import BeautifulSoup
import requests
import numpy as np
import urllib.parse
from google.cloud import bigquery
from google.oauth2 import service_account


# Define DAG function
@task()
def extract(pages):
    try:
        headers = ({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.54'})

        address = []
        beds = []
        baths = []
        areas = []
        prices = []

        web = ['https://www.trulia.com/NY/New_York/',
               'https://www.trulia.com/CA/Los_Angeles/',
               'https://www.trulia.com/IL/Chicago/',
               'https://www.trulia.com/AZ/Phoenix/',
               'https://www.trulia.com/NV/Las_Vegas/']

        for h in web:
            for i in range(1, pages + 1):
                website = requests.get(h + str(i) + '_p/', headers=headers)

                soup = BeautifulSoup(website.content, 'html.parser')

                result = soup.find_all('div',
                               {'data-testid': 'property-card-details'})

                result_update = [k for k in result if k.has_attr('data-testid')]

                for result in result_update:

                    try:
                        address.append(result.find('div', {'data-testid': 'property-address'}).get_text())
                    except:
                        address.append("n/a")

                    try:
                        beds.append(result.find('div', {'data-testid': 'property-beds'}).get_text())
                    except:
                        beds.append("n/a")

                    try:
                        baths.append(result.find('div', {'data-testid': 'property-baths'}).get_text())
                    except:
                        baths.append("n/a")

                    try:
                        areas.append(result.find('div', {'data-testid': 'property-floorSpace'}).get_text())
                    except:
                        areas.append("n/a")

                    try:
                        prices.append(result.find('div', {'data-testid': 'property-price'}).get_text())
                    except:
                        prices.append("n/a")

        real_estate_new = pd.DataFrame(list(zip(address, beds, baths, areas, prices)), columns=['Address', 'Beds', 'Baths', 'Area', 'Price'])
        df_dict = real_estate_new.to_dict('dict')
        return df_dict

    except Exception as e:
        print("Data extract error: " + str(e))


# Define DAG function
@task()
def transform(df_dict):
    try:
        df = pd.DataFrame.from_dict(df_dict)
        df['Beds'] = df['Beds'].str.lower()
        df['Beds'] = df['Beds'].apply(lambda x: x.strip('bd'))
        df['Baths'] = df['Baths'].apply(lambda x: x.strip('ba'))
        df['Price'] = df['Price'].apply(lambda x: x.strip('$'))
        df['Price'] = df['Price'].apply(lambda x: x.replace(",", ""))
        df['Price'] = df['Price'].apply(lambda x: x.replace("+", ""))
        df['Price'] = df['Price'].apply(lambda x: x.replace(".0", ""))
        df['Area'] = df['Area'].apply(lambda x: x.replace(" sqft", ""))

        df['Street'] = df['Address'].apply(lambda x: x.split(',')[0])
        df['District'] = df['Address'].apply(lambda x: x.split(',')[1])
        df['City'] = df['Address'].apply(lambda x: x.split(',')[2].split(' ')[1])
        df['Zip_Code'] = df['Address'].apply(lambda x: x.split(',')[2].split(' ')[2])

        df['Area'] = df['Area'].str.replace(',', '')
        df['Area'] = df['Area'].str.split(" ", n=1, expand=True)[0]

        df = df.where(df.Area != r'n/a')
        df = df.where(df.Area != r'nan')
        df = df.where(df.Baths != r'n/')
        df = df.where(df.Beds != r'studio')
        df = df.where(df.Price != r'')
        df = df.drop_duplicates()
        df = df.dropna()

        df['Area'] = df['Area'].astype('int')
        df['Beds'] = df['Beds'].astype('int')
        df['Baths'] = df['Baths'].astype('int')
        df['Price'] = df['Price'].astype('int')
        df['data_date'] = datetime.now(pytz.timezone("Asia/Jakarta")).date().strftime('%Y-%m-%d')
        df.insert(0, 'id', range(1, 1 + len(df)))
        df['id'] = df['id'].astype('str')
        df['uid'] = df['data_date'] + '_' + df['id'].astype(str)

        df_dict_clean = df.to_dict('dict')
        return df_dict_clean

    except Exception as e:
        print("Data transform error: " + str(e))


# Define DAG function
@task()
def load(df_dict_clean):
    try:
        credentials = service_account.Credentials.from_service_account_file(r"/home/dana123/airflow/latihan-345909-89e4eb39e2b1.json")
        project_id = 'latihan-345909'
        table_id = 'latihan-345909.real_estate.trulia'
        client = bigquery.Client(credentials=credentials, project=project_id)

        sql = """
       SELECT *
       FROM latihan-345909.real_estate.trulia
       """

        df = pd.DataFrame.from_dict(df_dict_clean)
        bq_df = client.query(sql).to_dataframe()

        changes = df[~df.apply(tuple, 1).isin(bq_df.apply(tuple, 1))]
        job = client.load_table_from_dataframe(changes, table_id)
        job.result()
        print("There are {0} rows added/changed".format(len(changes)))

    except Exception as e:
        print("Data load error: " + str(e))


# Declare Dag
with DAG(dag_id='etl_gcp',
         schedule_interval="0 0 * * *",
         start_date=datetime(2023, 5, 9),
         catchup=False,
         tags=['etl_gcp'])\
        as dag:
    src_data = extract(7)
    transform_src_data = transform(src_data)
    load_data = load(transform_src_data)

    src_data >> transform_src_data >> load_data




In [2]:
import time
from datetime import datetime
import pytz
import pandas as pd
import pandas_gbq
from bs4 import BeautifulSoup
import requests
import numpy as np
import urllib.parse
from google.cloud import bigquery
from google.oauth2 import service_account


# Define DAG function
def extract(pages):
    try:
        headers = ({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.54'})

        address = []
        beds = []
        baths = []
        areas = []
        prices = []

        web = ['https://www.trulia.com/NY/New_York/',
               'https://www.trulia.com/CA/Los_Angeles/',
               'https://www.trulia.com/IL/Chicago/',
               'https://www.trulia.com/AZ/Phoenix/',
               'https://www.trulia.com/NV/Las_Vegas/']

        for h in web:
            for i in range(1, pages + 1):
                website = requests.get(h + str(i) + '_p/', headers=headers)

                soup = BeautifulSoup(website.content, 'html.parser')

                result = soup.find_all('div',
                               {'data-testid': 'property-card-details'})

                result_update = [k for k in result if k.has_attr('data-testid')]

                for result in result_update:

                    try:
                        address.append(result.find('div', {'data-testid': 'property-address'}).get_text())
                    except:
                        address.append("n/a")

                    try:
                        beds.append(result.find('div', {'data-testid': 'property-beds'}).get_text())
                    except:
                        beds.append("n/a")

                    try:
                        baths.append(result.find('div', {'data-testid': 'property-baths'}).get_text())
                    except:
                        baths.append("n/a")

                    try:
                        areas.append(result.find('div', {'data-testid': 'property-floorSpace'}).get_text())
                    except:
                        areas.append("n/a")

                    try:
                        prices.append(result.find('div', {'data-testid': 'property-price'}).get_text())
                    except:
                        prices.append("n/a")

        real_estate_new = pd.DataFrame(list(zip(address, beds, baths, areas, prices)), columns=['Address', 'Beds', 'Baths', 'Area', 'Price'])
        df_dict = real_estate_new.to_dict('dict')
        return df_dict

    except Exception as e:
        print("Data extract error: " + str(e))


# Define DAG function
def transform(df_dict):
    try:
        df = pd.DataFrame.from_dict(df_dict)
        df['Beds'] = df['Beds'].str.lower()
        df['Beds'] = df['Beds'].apply(lambda x: x.strip('bd'))
        df['Baths'] = df['Baths'].apply(lambda x: x.strip('ba'))
        df['Price'] = df['Price'].apply(lambda x: x.strip('$'))
        df['Price'] = df['Price'].apply(lambda x: x.replace(",", ""))
        df['Price'] = df['Price'].apply(lambda x: x.replace("+", ""))
        df['Price'] = df['Price'].apply(lambda x: x.replace(".0", ""))
        df['Area'] = df['Area'].apply(lambda x: x.replace(" sqft", ""))

        df['Street'] = df['Address'].apply(lambda x: x.split(',')[0])
        df['District'] = df['Address'].apply(lambda x: x.split(',')[1])
        df['City'] = df['Address'].apply(lambda x: x.split(',')[2].split(' ')[1])
        df['Zip_Code'] = df['Address'].apply(lambda x: x.split(',')[2].split(' ')[2])

        df['Area'] = df['Area'].str.replace(',', '')
        df['Area'] = df['Area'].str.split(" ", n=1, expand=True)[0]

        df = df.where(df.Area != r'n/a')
        df = df.where(df.Area != r'nan')
        df = df.where(df.Baths != r'n/')
        df = df.where(df.Beds != r'studio')
        df = df.where(df.Price != r'')
        df = df.drop_duplicates()
        df = df.dropna()

        df['Area'] = df['Area'].astype('int')
        df['Beds'] = df['Beds'].astype('int')
        df['Baths'] = df['Baths'].astype('int')
        df['Price'] = df['Price'].astype('int')
        df['data_date'] = datetime.now(pytz.timezone("Asia/Jakarta")).date().strftime('%Y-%m-%d')
        df.insert(0, 'id', range(1, 1 + len(df)))
        df['id'] = df['id'].astype('str')
        df['uid'] = df['data_date'] + '_' + df['id'].astype(str)

        df_dict_clean = df.to_dict('dict')
        return df_dict_clean

    except Exception as e:
        print("Data transform error: " + str(e))


# Define DAG function
def load(df_dict_clean):
    try:
        credentials = service_account.Credentials.from_service_account_file(r"/home/dana123/airflow/latihan-345909-89e4eb39e2b1.json")
        project_id = 'latihan-345909'
        table_id = 'latihan-345909.real_estate.trulia'
        client = bigquery.Client(credentials=credentials, project=project_id)

        sql = """
       SELECT *
       FROM latihan-345909.real_estate.trulia
       """

        df = pd.DataFrame.from_dict(df_dict_clean)
        bq_df = client.query(sql).to_dataframe()

        changes = df[~df.apply(tuple, 1).isin(bq_df.apply(tuple, 1))]
        job = client.load_table_from_dataframe(changes, table_id)
        job.result()
        print("There are {0} rows added/changed".format(len(changes)))

    except Exception as e:
        print("Data load error: " + str(e))






In [3]:
test = extract(1)
test

{'Address': {0: '37-20 Prince St   #6C, Flushing, NY 11354',
  1: '1447 81st St, Brooklyn, NY 11228',
  2: '3202 Nostrand Ave #6E, Brooklyn, NY 11229',
  3: '474 W  238th St   #1B2B, Bronx, NY 10463',
  4: '301 E  62nd St   #12D, New York, NY 10065',
  5: '11050 71st Rd #2F, Flushing, NY 11375',
  6: '363 14th St #A, Brooklyn, NY 11215',
  7: '3728 Mayfair Dr, Los Angeles, CA 90065',
  8: '1325 Thayer Ave, Los Angeles, CA 90024',
  9: '11250 Wentworth St, Sun Valley, CA 91352',
  10: '13200 Pacific Promenade #450, Playa Vista, CA 90094',
  11: '17411 Hiawatha St, Granada Hills, CA 91344',
  12: '14171 Pinney St, Pacoima, CA 91331',
  13: '4656 Don Miguel Dr, Los Angeles, CA 90008',
  14: '7921 S  Marquette Ave, Chicago, IL 60617',
  15: '50 E  16th St #1213, Chicago, IL 60616',
  16: '1101 S  State St   #2307, Chicago, IL 60605',
  17: '1613 S  Central Park Ave, Chicago, IL 60623',
  18: '4332 N  St Louis Ave, Chicago, IL 60618',
  19: '4925 W  Rice St, Chicago, IL 60651',
  20: '1515 

In [38]:
df = pd.DataFrame.from_dict(test)
df['Beds'] = df['Beds'].str.lower()
df['Beds'] = df['Beds'].apply(lambda x: x.strip('bd'))
df['Beds'] = df['Beds'].str.strip()
df['Baths'] = df['Baths'].apply(lambda x: x.strip('ba'))
df['Baths'] = df['Baths'].str.strip()
df['Price'] = df['Price'].apply(lambda x: x.strip('$'))
df['Price'] = df['Price'].apply(lambda x: x.replace(",", ""))
df['Price'] = df['Price'].apply(lambda x: x.replace("+", ""))
df['Price'] = df['Price'].apply(lambda x: x.replace(".0", ""))
df['Price'] = df['Price'].str.strip()
df['Area'] = df['Area'].apply(lambda x: x.replace(" sqft", ""))
df['Area'] = df['Area'].str.strip()
df['Address']= df['Address'].str.strip()
df['Street'] = df['Address'].apply(lambda x: x.split(',')[0])
df['District'] = df['Address'].apply(lambda x: x.split(',')[1])
df['City'] = df['Address'].apply(lambda x: x.split(',')[2].split(' ')[1])
df['Zip_Code'] = df['Address'].apply(lambda x: x.split(',')[2].split(' ')[2])

df['Area'] = df['Area'].str.replace(',', '')
df['Area'] = df['Area'].str.split(" ", n=1, expand=True)[0]

df = df.where(df.Area != r'n/a')
df = df.where(df.Area != r'nan')
df = df.where(df.Area != r'')
df = df.where(df.Baths != r'n/')
df = df.where(df.Baths != r'')
df = df.where(df.Beds != r'studio')
df = df.where(df.Beds != r'')
df = df.where(df.Price != r'')
df = df.drop_duplicates()
df = df.dropna()

df['Area'] = df['Area'].astype('int64')
df['Beds'] = df['Beds'].astype('int64')
df['Baths'] = df['Baths'].astype('int64')
df['Price'] = df['Price'].astype('int64')
df['data_date'] = datetime.now(pytz.timezone("Asia/Jakarta")).date().strftime('%Y-%m-%d')
df.insert(0, 'id', range(1, 1 + len(df)))
df['id'] = df['id'].astype('str')
df['uid'] = df['data_date'] + '_' + df['id'].astype(str)

df['data_date']= df['data_date'].str.strip()
df['uid']= df['uid'].str.strip()

df_dict_clean = df.to_dict('dict')

In [39]:
df.head()

,id,Address,Beds,Baths,Area,Price,Street,District,City,Zip_Code,data_date,uid
0,1,"37-20 Prince St #6C, Flushing, NY 11354",2,1,790,720000,37-20 Prince St #6C,Flushing,NY,11354,2023-05-12,2023-05-12_1
1,2,"1447 81st St, Brooklyn, NY 11228",3,3,1368,1318888,1447 81st St,Brooklyn,NY,11228,2023-05-12,2023-05-12_2
2,3,"3202 Nostrand Ave #6E, Brooklyn, NY 11229",1,1,850,299000,3202 Nostrand Ave #6E,Brooklyn,NY,11229,2023-05-12,2023-05-12_3
3,4,"474 W 238th St #1B2B, Bronx, NY 10463",4,4,1800,735000,474 W 238th St #1B2B,Bronx,NY,10463,2023-05-12,2023-05-12_4
4,5,"301 E 62nd St #12D, New York, NY 10065",1,1,550,595000,301 E 62nd St #12D,New York,NY,10065,2023-05-12,2023-05-12_5


In [40]:
df.Price.unique()

array([ 720000, 1318888,  299000,  735000,  595000, 3275000,  898000,
       1300000,  835000, 1299000,  849999, 1599000,  374900,  259999,
       1499900,  485000,  479900,  279900,  690990,  360000,  415000,
        420000,  634000,  450000,  465000,  205000, 1400000], dtype=int64)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 34
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         27 non-null     object        
 1   Address    27 non-null     object        
 2   Beds       27 non-null     int64         
 3   Baths      27 non-null     int64         
 4   Area       27 non-null     int64         
 5   Price      27 non-null     int64         
 6   Street     27 non-null     object        
 7   District   27 non-null     object        
 8   City       27 non-null     object        
 9   Zip_Code   27 non-null     object        
 10  data_date  27 non-null     datetime64[ns]
 11  uid        27 non-null     object        
dtypes: datetime64[ns](1), int64(4), object(7)
memory usage: 2.7+ KB


In [42]:
credentials = service_account.Credentials.from_service_account_file(r"latihan-345909-89e4eb39e2b1.json")
project_id = 'latihan-345909'
table_id = 'latihan-345909.real_estate.trulia'
client = bigquery.Client(credentials=credentials, project=project_id)

sql = """
SELECT *
FROM latihan-345909.real_estate.trulia
"""

df = pd.DataFrame.from_dict(df_dict_clean)
df['data_date'] = df['data_date'].apply(pd.to_datetime)
bq_df = client.query(sql).to_dataframe()

changes = df[~df.apply(tuple, 1).isin(bq_df.apply(tuple, 1))]
job = client.load_table_from_dataframe(changes, table_id)
job.result()
print("There are {0} rows added/changed".format(len(changes)))

There are 27 rows added/changed


In [10]:
for i in df_dict_clean.keys():
    print(type(df_dict_clean[i][0]))

<class 'str'>
<class 'str'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [29]:
bq_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3433 entries, 0 to 3432
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         3433 non-null   object
 1   address    3433 non-null   object
 2   beds       3433 non-null   Int64 
 3   baths      3433 non-null   Int64 
 4   area       3433 non-null   Int64 
 5   price      3433 non-null   Int64 
 6   street     3433 non-null   object
 7   district   3433 non-null   object
 8   city       3433 non-null   object
 9   zip_code   3433 non-null   object
 10  data_date  3433 non-null   dbdate
 11  uid        3433 non-null   object
dtypes: Int64(4), dbdate(1), object(7)
memory usage: 335.4+ KB


In [ ]:
pandas